In [1]:
import numpy as np
import pandas as pd
import json
from random import shuffle, sample
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [2]:
train_df = pd.read_json('data/train.jsonl', lines=True)
train_df.head()

,messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,players,game_id
0,[Germany!\n\nJust the person I want to speak w...,"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, NOANNOTATION, NOANNOT...","[italy, germany, italy, germany, italy, italy,...","[germany, italy, germany, italy, germany, germ...","[74, 76, 86, 87, 89, 92, 97, 117, 119, 121, 12...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Spring, Sprin...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[italy, germany]",1
1,[Hello there! What's your general plan for thi...,"[True, False, True, False, True, True, True, T...","[True, True, True, True, True, NOANNOTATION, T...","[austria, italy, austria, italy, italy, austri...","[italy, austria, italy, austria, austria, ital...","[1, 67, 71, 73, 98, 99, 101, 179, 181, 185, 18...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Spring, Sprin...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, -1, -...","[italy, austria]",1
2,[Buongiorno! \nBe kinda nice to know if you're...,"[True, True, False, True, True, True, True, Tr...","[True, False, True, False, True, True, NOANNOT...","[russia, italy, russia, italy, russia, italy, ...","[italy, russia, italy, russia, italy, russia, ...","[11, 50, 52, 57, 61, 66, 77, 85, 96, 102, 116,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Spring, Sprin...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[4, 3, 4, 3, 4, 3, 4, 3, 3, 3, 4, 3, 3, 4, 4, ...","[1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1...","[italy, russia]",1
3,[Hey italy! good luck this game. I'm guessing ...,"[True, False, True, True, True, True, True, Tr...","[NOANNOTATION, True, True, False, True, True, ...","[england, italy, england, england, england, it...","[italy, england, italy, italy, italy, england,...","[32, 95, 106, 107, 108, 110, 113, 125, 126, 12...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Spring, Sprin...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[italy, england]",1
4,[Hello Italy what’s up what are your thoughts ...,"[True, False, False, True, True, True, True, T...","[NOANNOTATION, True, True, True, True, True, N...","[turkey, italy, italy, italy, turkey, italy, t...","[italy, turkey, turkey, turkey, italy, turkey,...","[45, 94, 103, 150, 154, 178, 192, 194, 195, 19...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[Spring, Spring, Spring, Spring, Fall, Fall, F...","[1901, 1901, 1901, 1901, 1901, 1901, 1901, 190...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1...","[italy, turkey]",1


In [3]:
def to_single_message_format(gamefile):
    messages = []
    with open(gamefile) as inh:
        for ln in inh:
            conversation = json.loads(ln)
            for msg, sender_label, receiver_label, score_delta \
                in zip(conversation['messages'],conversation['sender_labels'], \
                    conversation['receiver_labels'], conversation['game_score_delta']):
                messages.append({'message': msg, 'receiver_annotation': receiver_label,\
                    'sender_annotation':sender_label, 'score_delta': int(score_delta)})
    shuffle(messages)    
    return messages

In [4]:
def write_single_messages(messages, outfile):
    with open(outfile, "w") as outh:
        for msg in messages:
            outh.write(json.dumps(msg)+'\n')

In [16]:
ROOT = "data"
write_single_messages(to_single_message_format(os.path.join(ROOT, 'validation.jsonl')) , os.path.join(ROOT, 'validation_sm.jsonl'))
write_single_messages(to_single_message_format(os.path.join(ROOT, 'train.jsonl')) , os.path.join(ROOT, 'train_sm.jsonl'))
write_single_messages(to_single_message_format(os.path.join(ROOT, 'test.jsonl')) ,  os.path.join(ROOT, 'test_sm.jsonl'))

---

In [12]:
# Load the JSONL data
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Preprocess the data
def preprocess_data(df):
    df['label'] = df['score_delta'].apply(lambda x: 1 if x < 0 else 0) 
    return df


train_data = preprocess_data(load_data("data/train_sm.jsonl"))
test_data = preprocess_data(load_data("data/test_sm.jsonl"))
validation_data = preprocess_data(load_data("data/validation_sm.jsonl"))

print(train_data.head())

                                             message receiver_annotation  \
0  I see! Do you see an issue with me taking denm...                True   
1                                   Okay let me know                True   
2  Rgr.  Stand ready to support whatever you decide.                True   
3       Sidebar- what’re you gonna do about england?        NOANNOTATION   
4  Yea I’m here. I’m with you on that. Not cuttin...                True   

   sender_annotation  score_delta  label  
0               True            0      0  
1               True           -1      1  
2               True           -2      1  
3               True           -3      1  
4               True           -2      1  


In [13]:
vectorizer = TfidfVectorizer(max_features=500)
X_train_text = vectorizer.fit_transform(train_data['message'])
X_test_text = vectorizer.transform(test_data['message'])
X_validation_text = vectorizer.transform(validation_data['message'])

y_train = train_data['label']
y_test = test_data['label']
y_validation = validation_data['label']


model = LogisticRegression()
model.fit(X_train_text, y_train)


y_test_pred = model.predict(X_test_text)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test, y_test_pred))


y_validation_pred = model.predict(X_validation_text)
print("Validation Accuracy:", accuracy_score(y_validation, y_validation_pred))
print("Validation Classification Report:\n", classification_report(y_validation, y_validation_pred))

Test Accuracy: 0.6847865742429771
Test Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.96      0.81      1901
           1       0.40      0.05      0.10       840

    accuracy                           0.68      2741
   macro avg       0.55      0.51      0.45      2741
weighted avg       0.61      0.68      0.59      2741

Validation Accuracy: 0.7161016949152542
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.97      0.83      1022
           1       0.41      0.05      0.08       394

    accuracy                           0.72      1416
   macro avg       0.57      0.51      0.46      1416
weighted avg       0.64      0.72      0.62      1416

